# LangChain #2 - 개발에 도움이 되는 정보

* API 사용금액 조회
* 한글 Summarization
* Python 객체와 친해지기
* PromptLayer 서비스
---
참고 자료:
* 파이썬 문서: https://python.langchain.com/en/latest/index.html
* 개념 문서: https://docs.langchain.com/docs/
* 소스: https://github.com/hwchase17/langchain
---

In [ ]:
#@title 기본 패키지(openai, langchain) 설치
!pip install -q openai
!pip install -q langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.3/70.3 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 269.3/269.3 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 626.5/626.5 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 57.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 3.5 MB/s eta 0:00:00


In [ ]:
#@title 기타 패키지 설치 (구글검색, 위키피디아, VectorStore, HuggingFace Embedding)
!pip install -q google-search-results
!pip install -q wikipedia
!pip install -q faiss-cpu # 오픈소스 벡터DB (Facebook, MIT license)
!pip install -q sentence_transformers # HuggingFace Embedding 사용 위해서 필요
!pip install -q tiktoken # Summarization 할때 필요
!pip install -q promptlayer # openai API 조회 서비스

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 73.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 4.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 62.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 56.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 21.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 76.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 17.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [ ]:
import os
#@markdown https://platform.openai.com/account/api-keys
OPENAI_API_KEY = "" #@param {type:"string"}

# #@markdown https://huggingface.co/settings/tokens
# #@markdown HuggingFace에서 모델 다운로드나 클라우드 모델 사용하기 위해서 필요 (무료)
# HUGGINGFACEHUB_API_TOKEN = "" #@param {type:"string"}

# #@markdown https://serpapi.com/manage-api-key
# #@markdown 구글 검색하기 위해서 필요 (월 100회 무료)
# SERPAPI_API_KEY = "" #@param {type:"string"}

#@markdown https://promptlayer.com/
#@markdown OpenAI API/프롬프트를 내역을 로그로 남기는 서비스
PROMPTLAYER_API_KEY = "" #@param {type:"string"}

os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
# os.environ["HUGGINGFACEHUB_API_TOKEN"] = HUGGINGFACEHUB_API_TOKEN
# os.environ["SERPAPI_API_KEY"] = SERPAPI_API_KEY
os.environ["PROMPTLAYER_API_KEY"] = PROMPTLAYER_API_KEY


In [ ]:
#@title 1. API 사용 금액 조회

from langchain.llms import OpenAI
from langchain.callbacks import get_openai_callback

chat = OpenAI(model_name='gpt-3.5-turbo')


# chat("1980년대 메탈 음악 5곡 추천해줘.")
with get_openai_callback() as cb:

  result = chat("1980년대 메탈 음악 5곡 추천해줘.")

  print(f"Total Tokens: {cb.total_tokens}")
  print(f"Prompt Tokens: {cb.prompt_tokens}")
  print(f"Completion Tokens: {cb.completion_tokens}")
  print(f"Total Cost (USD): ${cb.total_cost}")
  print(cb)

result

/usr/local/lib/python3.9/dist-packages/langchain/llms/openai.py:165: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/langchain/llms/openai.py:672: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


Total Tokens: 92
Prompt Tokens: 31
Completion Tokens: 61
Total Cost (USD): $0.000184
Tokens Used: 92
	Prompt Tokens: 31
	Completion Tokens: 61
Successful Requests: 1
Total Cost (USD): $0.000184


'1. "Master of Puppets" - Metallica\n2. "Crazy Train" - Ozzy Osbourne\n3. "Thunderstruck" - AC/DC\n4. "Peace Sells" - Megadeth\n5. "Heaven and Hell" - Black Sabbath'

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

chat = ChatOpenAI()

with get_openai_callback() as cb:
  result = chat([HumanMessage(content="1990년대 메탈 음악 5곡 추천해줘.")])
  print(cb)
result

Tokens Used: 0
	Prompt Tokens: 0
	Completion Tokens: 0
Successful Requests: 0
Total Cost (USD): $0.0


AIMessage(content='1. Metallica - "Enter Sandman"\n2. Pantera - "Walk"\n3. Nirvana - "Smells Like Teen Spirit"\n4. Guns N\' Roses - "November Rain"\n5. Iron Maiden - "Fear of the Dark"', additional_kwargs={})

In [ ]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

chain = LLMChain(llm=chat, prompt=PromptTemplate(input_variables=["text"], template="{text}"))


with get_openai_callback() as cb:
  result = chain.run("2000년대 메탈 음악 5곡 추천해줘.")
  print(cb)
result

Tokens Used: 85
	Prompt Tokens: 31
	Completion Tokens: 54
Successful Requests: 1
Total Cost (USD): $0.00017


'1. Slipknot - Wait and Bleed\n2. Avenged Sevenfold - Bat Country\n3. Disturbed - Down with the Sickness\n4. System of a Down - Chop Suey!\n5. Lamb of God - Laid to Rest'

In [ ]:
#@title 2. 한글 Summarization

from langchain.chat_models import ChatOpenAI

chat = ChatOpenAI()

from langchain.document_loaders import WebBaseLoader

loader = WebBaseLoader(web_path="https://ko.wikipedia.org/wiki/NewJeans")
documents = loader.load()

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)
len(docs)

12

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.chains.summarize import load_summarize_chain
chain = load_summarize_chain(chat, chain_type="map_reduce", verbose=True)

In [ ]:
chain.llm_chain.prompt.template

'Write a concise summary of the following:\n\n\n"{text}"\n\n\nCONCISE SUMMARY:'

In [ ]:
chain.llm_chain.prompt.template = chain.llm_chain.prompt.template.replace("CONCISE SUMMARY:", "CONCISE SUMMARY IN KOREAN:")

In [ ]:
chain.llm_chain.prompt.template

'Write a concise summary of the following:\n\n\n"{text}"\n\n\nCONCISE SUMMARY IN KOREAN:'

In [ ]:
with get_openai_callback() as cb:
  result = chain.run(docs[:3])
  print(cb)
result



> Entering new MapReduceDocumentsChain chain...
Prompt after formatting:
Write a concise summary of the following:


"NewJeans - 위키백과, 우리 모두의 백과사전






































본문으로 이동








주 메뉴





주 메뉴
사이드바로 이동
숨기기



		둘러보기
	

대문최근 바뀜요즘 화제임의의 문서로기부




		사용자 모임
	

사랑방사용자 모임관리 요청




		편집 안내
	

도움말정책과 지침질문방




언어

이 위키백과에서 언어 링크는 문서 제목 건너편의 문서 최상단에 있습니다. 최상단으로 이동합니다.



















검색















계정 만들기로그인






개인 도구




 계정 만들기 로그인




		로그아웃한 편집자를 위한 문서 더 알아보기


기여토론


























목차
사이드바로 이동
숨기기




처음 위치





1활동


				활동 하위섹션 토글하기
			




1.1데뷔 전







1.22022년









2구성원







3역사


				역사 하위섹션 토글하기
			




3.12019–2021: 결성 및 데뷔 전 활동









4음반


				음반 하위섹션 토글하기
			




4.1EP







4.2싱글









5음반 외 활동







6수상 목록


				수상 목록 하위섹션 토글하기
			




6.1가요 프로그램 1위









7기타 시상식







8각주







9외부 링크












						목차 토글
					





목차 토글







NewJeans




20개 언어



العربيةБеларускаяEnglishEspañolفارسیSuomiFrançaisՀայերենBahasa

"NewJeans은 대한민국의 5인조 록밴드로, 2019년 결성되어 EP와 싱글을 발표하며 활동하고 있으며 가요 프로그램에서 1위를 수상한 적이 있고 현재까지 수상 목록을 가지고 있다. 또한, 2022년 7월 22일에 데뷔한 대한민국의 5인조 걸 그룹으로, 어도어 레이블에서 활동하며, 2022년 8월 18일에는 엠카운트다운에서 데뷔 3주만에 첫 1위를 차지했다. 민지와 하니는 방탄소년단의 Permission to Dance 뮤직비디오에 출연하였다. 2022년 8월 8일에는 정식 앨범 'New Jeans'를 발매했다."

In [ ]:
#@title 3. Python Object와 친해지기

chain
# str(chain)
# chain.__str__()
# chain.__repr__()

MapReduceDocumentsChain(memory=None, callback_manager=<langchain.callbacks.shared.SharedCallbackManager object at 0x7fef9ebd7e20>, verbose=True, input_key='input_documents', output_key='output_text', llm_chain=LLMChain(memory=None, callback_manager=<langchain.callbacks.shared.SharedCallbackManager object at 0x7fef9ebd7e20>, verbose=True, prompt=PromptTemplate(input_variables=['text'], output_parser=None, partial_variables={}, template='Write a concise summary of the following:\n\n\n"{text}"\n\n\nCONCISE SUMMARY IN KOREAN:', template_format='f-string', validate_template=True), llm=ChatOpenAI(verbose=False, callback_manager=<langchain.callbacks.shared.SharedCallbackManager object at 0x7fef9ebd7e20>, client=<class 'openai.api_resources.chat_completion.ChatCompletion'>, model_name='gpt-3.5-turbo', temperature=0.7, model_kwargs={}, openai_api_key=None, openai_organization=None, request_timeout=60, max_retries=6, streaming=False, n=1, max_tokens=None), output_key='text'), combine_document_ch

In [ ]:
dir(chain)

In [ ]:
chain.__repr_args__()

[('memory', None),
 ('callback_manager',
  <langchain.callbacks.shared.SharedCallbackManager at 0x7fef9ebd7e20>),
 ('verbose', True),
 ('input_key', 'input_documents'),
 ('output_key', 'output_text'),
 ('llm_chain',
  LLMChain(memory=None, callback_manager=<langchain.callbacks.shared.SharedCallbackManager object at 0x7fef9ebd7e20>, verbose=True, prompt=PromptTemplate(input_variables=['text'], output_parser=None, partial_variables={}, template='Write a concise summary of the following:\n\n\n"{text}"\n\n\nCONCISE SUMMARY IN KOREAN:', template_format='f-string', validate_template=True), llm=ChatOpenAI(verbose=False, callback_manager=<langchain.callbacks.shared.SharedCallbackManager object at 0x7fef9ebd7e20>, client=<class 'openai.api_resources.chat_completion.ChatCompletion'>, model_name='gpt-3.5-turbo', temperature=0.7, model_kwargs={}, openai_api_key=None, openai_organization=None, request_timeout=60, max_retries=6, streaming=False, n=1, max_tokens=None), output_key='text')),
 ('combine

In [ ]:
chain.llm_chain.prompt.__repr_args__()

[('input_variables', ['text']),
 ('output_parser', None),
 ('partial_variables', {}),
 ('template',
  'Write a concise summary of the following:\n\n\n"{text}"\n\n\nCONCISE SUMMARY IN KOREAN:'),
 ('template_format', 'f-string'),
 ('validate_template', True)]

In [ ]:
#@title 4. PromptLayer 서비스

from langchain.chat_models import PromptLayerChatOpenAI
# chat = ChatOpenAI()
chat = PromptLayerChatOpenAI(pl_tags=["langchain_230424"], temperature=0.9)

chat([HumanMessage(content="1980년대 메탈 음악 10곡 추천해줘.")])

AIMessage(content='1. "Back in Black" - AC/DC\n2. "Master of Puppets" - Metallica\n3. "Paradise City" - Guns N\' Roses\n4. "Crazy Train" - Ozzy Osbourne\n5. "The Trooper" - Iron Maiden\n6. "Breaking the Law" - Judas Priest\n7. "Holy Diver" - Dio\n8. "Ace of Spades" - Motörhead\n9. "Eruption/You Really Got Me" - Van Halen\n10. "Raining Blood" - Slayer', additional_kwargs={})